In [42]:
%pip install docker
import yaml
import docker
import re
import subprocess
from concurrent.futures import ThreadPoolExecutor
from time import sleep

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
def parse_yaml_script_file(file_path):
    with open(file_path) as f:
        try:
            text = f.read()
            parsed_data = yaml.safe_load(text)
            return parsed_data
        except FileNotFoundError:
            print('File not found')
            return None
        except yaml.YAMLError as exc:
            print(exc)
            return None

In [44]:
def process_steps(steps, initialVariables,variables, job):
    for step in steps:
        if 'script' in step:
            script_content = step['script']
            expanded_script = expand_variables(script_content, variables, initialVariables)
            execute_script(expanded_script)
        elif 'bash' in step:
            bash_command = step['bash']
            expanded_bash_command = expand_variables(bash_command, variables, initialVariables)
            print(f"Executing bash command: {expanded_bash_command}\n")
            run_bash(expanded_bash_command, variables)
        else:
            print("Unsupported step type")
    return job

def expand_variables(content, variables, initialVariables):
    for key in variables:
        content = re.sub(fr'\${{\s*{{\s*variables.{key}\s*}}\s*}}', str(initialVariables[key]), content)
        content = re.sub(fr"\$\(\s*{key}\s*\)", str(variables[key]), content)
    return content

def run_bash(bash_command, variables):
    match = re.search(r"\[task\.setvariable variable=(\w+)\](\w+)", bash_command)
    if match:
        print(f"---------------------Updating variable {match.group(1)} to {match.group(2)}----------------------")
        variable_name = match.group(1)
        new_value = match.group(2)
        for key in variables:
            if key == variable_name:
                variables[key] = new_value

image_name = "my_image"
dockerfile_path = "."

def execute_script(script):
    client = docker.from_env()

    try:
        # Build the Docker image
        print(f"Building Docker image '{image_name}'...")
        client.images.build(path=dockerfile_path, tag=image_name, rm=True)
        script_new = script.replace("\n", ";")

        # Run a bash command in a new container based on the built image
        print(f"Executing bash command '{script_new}' in a container based on '{image_name}'...")
        container = client.containers.run(image_name, command=['/bin/bash', '-c', script_new], detach=True)

        # Capture and print the container output
        output = container.logs().decode('utf-8')
        print(f"Output of the command:\n{output}")

        # Stop and remove the container
        container.stop()
        container.remove()

    except docker.errors.BuildError as e:
        print(f"Error building Docker image: {e}")
    except docker.errors.APIError as e:
        print(f"Error interacting with Docker: {e}")

def execute_script_without_docker(script):
    for line in script.split('\n'):
        result = subprocess.run(line, shell=True, capture_output=True, text=True)
        

def run_docker_command(command):
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout, result.stderr, result.returncode


In [45]:
yaml_file_path = 'test.yml'
parsed_data = parse_yaml_script_file(yaml_file_path)

def is_job_complete(completed_jobs, job_name):
    for job in completed_jobs:
        if job.get('displayName') == job_name:
            return True
    return False

if(parsed_data):
    initialVariables = {}
    for key in parsed_data.get('variables', []):
        initialVariables[key.get('name')] = key.get('value')
    variables = initialVariables.copy()

    jobs = parsed_data.get('jobs', [])
    completed_jobs = []
    started_jobs = jobs.copy()
    
    
    print(parsed_data)
    with ThreadPoolExecutor() as executor:
        futures = []
        while len(completed_jobs) != len(jobs):
            print("Checking")
            print(started_jobs)
            ids_jobs = []
            ids_futures = []
            for idx, job in enumerate(started_jobs):
                job_name = job.get("displayName", [])
                job_condition = not job.get('dependsOn') or is_job_complete(completed_jobs, job.get('dependsOn'))
                print(f"Job {job_name}: {job_condition}")
                if not job.get('dependsOn') or is_job_complete(completed_jobs, job.get('dependsOn')):
                    print("Executing job: " + job.get("displayName", []))
                    steps = job.get('steps', [])
                    futures.append(executor.submit(process_steps, steps, initialVariables, variables, job))
                    ids_jobs.append(idx)
            
            for id in ids_jobs:
                started_jobs.pop(id)
            
            for idx, future in enumerate(futures):
                job = future.result()
                print(f"Completed job {job.get('displayName')}")
                completed_jobs.append(job)
                ids_futures.append(idx)
            
            print(ids_futures)
            print(futures)
            
            for id in ids_futures:
                futures.pop(id)

{'parameters': [{'name': 'image', 'displayName': 'Pool Image', 'type': 'string', 'default': 'ubuntu-latest', 'values': ['windows-latest', 'ubuntu-latest', 'macOS-latest']}], 'variables': [{'name': 'one', 'value': 'initialValue'}, {'name': 'two', 'value': 'initialValue'}], 'jobs': [{'job': 'build', 'displayName': 'build', 'pool': {'vmImage': '${{ parameters.image }}'}, 'steps': [{'script': 'echo ${ { variables.one  }  }\necho $(one)\n', 'displayName': 'First variable pass'}, {'bash': 'echo "##vso[task.setvariable variable=one]secondValue"', 'displayName': 'Set new variable value'}, {'script': 'echo ${ { variables.one } }\necho $(one)\n', 'displayName': 'Second variable pass'}]}, {'job': 'build2', 'displayName': 'build2', 'pool': {'vmImage': '${{ parameters.image }}'}, 'steps': [{'script': 'echo ${ { variables.one  }  }\necho $(one)\n', 'displayName': 'First variable pass'}, {'bash': 'echo "##vso[task.setvariable variable=one]secondValue"', 'displayName': 'Set new variable value'}, {'scr

IndexError: pop index out of range